# Machine Learning Project 2

In [38]:
# Imports
import pandas as pd
import numpy as np

---
## Data Loading

In this section we load the data for :
* positive tweets, label= `:)` ($1$ for classification) 
* negative tweets, label= `:(` ($-1$ for classification)

Full data is used below (1'250'000 tweets).

In [39]:
data_path = '../data/'

## Load full training sets
# positive
pos = pd.read_table(data_path+'train_pos_full.txt', sep='.\n', names=['tweet'], engine='python')
pos['label']=1
print(f"Loaded POS data, correctly interpreted 1-tweet-per-line fashion : {pos.shape[0]==1_250_000}")

# negative
neg = pd.read_table(data_path+'train_neg_full.txt', sep='.\n', names=['tweet'], engine='python')
neg['label']=-1
print(f"Loaded NEG data, correctly interpreted 1-tweet-per-line fashion : {neg.shape[0]==1_250_000}")

# Data sizes
print(f"Number of tweets : (POS) {pos.shape[0]} (NEG) {neg.shape[0]}\n")

# Merge datasets to get a complete training set
tweets = pos.append(neg)
tweets

Loaded POS data, correctly interpreted 1-tweet-per-line fashion : True
Loaded NEG data, correctly interpreted 1-tweet-per-line fashion : True
Number of tweets : (POS) 1250000 (NEG) 1250000



,tweet,label
0,<user> i dunno justin read my mention or not ....,1
1,"because your logic is so dumb , i won't even c...",1
2,""" <user> just put casper in a box ! "" looved t...",1
3,<user> <user> thanks sir > > don't trip lil ma...,1
4,visiting my brother tmr is the bestest birthda...,1
...,...,...
1249995,im so sorry ! <user> & to <user> & <user> u gu...,-1
1249996,i can't find food coloring anywhere,-1
1249997,<user> same here ! ! but tort ! ! wonder why y...,-1
1249998,keyless entry remote fob clicker for 2005 buic...,-1


In [40]:
## Load word embeddings and vocabulary to compute word vectors of tweets

# Load word embeddings
embeddings = np.load(data_path + 'embeddings_full.npy')
print(f'Loaded word embeddings in structure of type {type(embeddings)}.')

# Loading vocab
words = pd.read_table(data_path + 'vocab_cut.txt', sep='.\n', names=['word'], engine='python', squeeze=True, na_values=np.nan)
print(f'Loaded word embeddings in structure of type {type(words)}.')

# Check that the vocabulary encompasses all embedded words
print(f'\nBoth the embeddings and the vocabulary are same length :  {len(embeddings)==words.shape[0]}')
print(f"Embeddings: {embeddings.shape}, vocab: {words.shape}")

## Clean the data

# Drop NaN values in words
nas = words.isna()
words.dropna(inplace=True)
embeddings = np.delete(embeddings, nas[nas].index.values, axis=0)
print(f'NA values were dropped in both tables: {len(embeddings)==words.shape[0]}')
print(f"Embeddings: {embeddings.shape}, vocab: {words.shape}")

# Index by words for faster index-for-word search
words = pd.DataFrame(data=words.index, index=words.values)
embeddings = pd.DataFrame(embeddings, index=words.index)

Loaded word embeddings in structure of type <class 'numpy.ndarray'>.
Loaded word embeddings in structure of type <class 'pandas.core.series.Series'>.

Both the embeddings and the vocabulary are same length :  True
Embeddings: (101298, 20), vocab: (101298,)
NA values were dropped in both tables: True
Embeddings: (101296, 20), vocab: (101296,)


In [41]:
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
<user>,0.000327,0.028452,0.016525,0.000613,0.005188,-0.002815,0.009533,-0.044277,0.009413,0.031187,0.010009,-0.002514,0.007920,0.036842,-0.012512,0.022829,-0.000567,-0.016734,-0.008299,-0.008789
!,-0.019161,0.027524,0.033832,0.012091,-0.027132,-0.012628,-0.000841,-0.033120,0.013693,-0.004513,0.016704,0.015588,-0.008836,0.035705,0.007875,0.015618,0.009505,-0.011820,-0.003584,-0.005041
i,0.019321,0.030986,0.014057,0.007218,-0.002602,-0.022695,-0.004233,-0.033344,-0.001470,0.013007,0.002135,-0.007511,0.003676,0.010762,-0.002978,-0.001809,0.010544,-0.023608,-0.014181,-0.004194
the,0.002400,0.021290,0.007768,-0.005454,-0.016949,-0.001816,-0.008950,-0.037330,-0.029912,-0.008472,0.005187,-0.016195,0.006317,-0.010477,-0.022874,-0.001610,-0.006108,-0.008685,-0.011349,0.006130
.,-0.002507,0.011608,0.027699,0.013425,-0.020171,0.008285,-0.012358,-0.045863,-0.002258,0.031533,-0.014051,0.008234,0.008522,0.012892,0.017243,-0.006924,0.004531,-0.021841,-0.005605,-0.022420


In [42]:
words.head()

,0
<user>,0
!,1
i,2
the,3
.,4


---
## Exploratory Data Analysis
In this part we analyse our data in order to optimize its information.

### Cleaning tags 
Here we explore the non-spoken tags present in the tweets and determine if they are relevant for our sentiment analysis.

In [43]:
import re
from collections import defaultdict

## We check if the tags are relevant information between both pos and neg cases

def count_HTML_tags(series) :
    """
    Returns stats about the HTML tags in the tweet series.
    Returns :
    dic (defaultdict) : dict of all tags occurences.
    count (int) : count of all tags."""
    dic = defaultdict(lambda:0)
    def a(k):
        dic[k]+=1
        return None
    series.apply(lambda s : [a(k) for k in re.findall('<\/*[a-zA-Z]+>', s)])
    count = series.str.count('<\/*[a-zA-Z]+>').sum()
    return dic, count

# We query stats about the tags
d_pos, n_pos = count_HTML_tags(pos['tweet'])
d_neg, n_neg = count_HTML_tags(neg['tweet'])
all_keys = set(d_pos.keys()) | set(d_neg.keys())

print(f"|{'KEY':14s}|{'POS':6s}|{'NEG':6s}|")
for k in all_keys : 
    print(f"|{k:14s}|{d_pos[k]:6d}|{d_neg[k]:6d}|")

print(f"\nPOS tweets contain {n_pos} ({(n_pos-n_neg)*100/n_neg:.2f}%) HTML tags.")
print(f"NEG tweets contain {n_neg} ({(n_neg-n_pos)*100/n_pos:.2f}%) HTML tags.")

|KEY           |POS   |NEG   |
|</b>          |     1|    19|
|</span>       |     0|     1|
|</script>     |     0|     4|
|<thing>       |     0|     1|
|<screams>     |     0|     1|
|<sarah>       |     1|     0|
|<weirdarms>   |     1|     0|
|<sciencestuff>|     2|     0|
|<demon>       |     0|     1|
|</div>        |     0|     1|
|<br>          |     1|     6|
|<agent>       |     0|     3|
|<c>           |     1|     2|
|<strong>      |     0|     6|
|<content>     |     0|     1|
|<mournfully>  |     0|     1|
|<del>         |     0|     1|
|<g>           |     1|     0|
|<joke>        |     1|     0|
|<likewise>    |     1|     0|
|<brr>         |     1|     0|
|<ummm>        |     0|     1|
|<parenthood>  |     0|     1|
|</details>    |     1|     0|
|</em>         |     0|     2|
|</cfoutput>   |     0|     1|
|<please>      |     0|     1|
|<hugs>        |     1|     1|
|<emotional>   |     0|     1|
|<trans>       |     0|     3|
|<gardenstuff> |     2|     0|
|<w>    

**Note**: Although the difference in number of tags is not significant. The distribution of them is quite significant (i.e. for tags `<url>` and `<user>`). Thus we choose to leave the tags as part of the tweet. **THIS COULD BE REVIEWED TO IMPROVE PERF**

In [44]:
# Clean the HTML tags from the tweets
## CHANGE RETURN VAR IF RELEVANT

def clean_HTML_tags(series) :
    return series.str.replace('<\/*[a-zA-Z]+>', '', regex=True)

t = clean_HTML_tags(pos['tweet'])
t2 = clean_HTML_tags(neg['tweet'])

---
## Training
In this part we train the models on our data.
Thus, we perform
* a resampling of our data to work locally on a smaller set.
* the creation of word vectors for our tweets.
* a train-test-split to locally estimate the model's performance.
* cross-validation trainin on a series of models :
    * Linear Regression
    * Logistic Regression
    * SVM
    * Neural Networks

In [45]:
# Base parameters

# To compute size of local training set
local_t_size = 100_000
# location of precomputed files
word_vectors_file_location = '../precomputed_data/'
# indication on the number of dimensions of the embeddings that is being used
embeddings_dim_info = 20

### Resampling the Training set
Using only a set of 200'000 tweets locally to decrease computation time.

In [46]:
from sklearn.utils import resample

# Take only `0.5*local_t_size` samples from both classes for faster computation
n = int(local_t_size/2)
pos_ = resample(pos, n_samples=n, replace=False)
neg_ = resample(neg, n_samples=n, replace=False)
tweets_ = pos_.append(neg_)

tweets_

,tweet,label
757921,<user> <user> <user> as you can see carol they...,1
1196488,college park with <user> and <user> ayyy .,1
611562,<user> lol ! i told you i would look back and ...,1
352088,<user> well thanks,1
361024,<user> ha ha i like his tattoo's thats about i...,1
...,...,...
841954,trend spotting : pop star rita ora rocks monok...,-1
995160,i do not want to do this history essay such a ...,-1
241855,don't want to work,-1
1215945,<user> yeah she did and im doing that with you...,-1


### Word vectors creation

In [47]:
# Recall our data
print(f"Embeddings for {embeddings.shape[0]:,.1f} with {embeddings.shape[1]} features for each word.") 
print(f'Embeddings shape : {embeddings.shape}.\n')
embeddings.head(2)

Embeddings for 101,296.0 with 20 features for each word.
Embeddings shape : (101296, 20).



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
<user>,0.000327,0.028452,0.016525,0.000613,0.005188,-0.002815,0.009533,-0.044277,0.009413,0.031187,0.010009,-0.002514,0.007920,0.036842,-0.012512,0.022829,-0.000567,-0.016734,-0.008299,-0.008789
!,-0.019161,0.027524,0.033832,0.012091,-0.027132,-0.012628,-0.000841,-0.033120,0.013693,-0.004513,0.016704,0.015588,-0.008836,0.035705,0.007875,0.015618,0.009505,-0.011820,-0.003584,-0.005041


In [48]:
def word_vector(tweet):
    """
    Creates the feature vector corresponding to the tweet.
    To do so, computes the mean of the word embeddings corresponding to the vocabulary words in the tweet.
    
    Parameters 
    ----------
    tweet : str
        Input tweet from which the word vector is created.
    """
    split_by_words = tweet.split()
    embed_list = []
    
    # Get vocab word embeddings
    for w in split_by_words:
        if w in words.index :
            embed_list.append(  embeddings.loc[w].values  )
        
    # Compute mean if any vocab word was found
    mean = np.zeros(20) if not embed_list else np.mean(embed_list, axis=0) 
    return mean.tolist()

In [49]:
%%time
from pandas.errors import MergeError

try :
    # Load pre-computed word vectors file if possible 
    name = word_vectors_file_location+'word_vectors_'+str(local_t_size)+'_'+str(embeddings_dim_info)+'.npy'
    print(f"Trying to load word vectors from {name}")
    precomputed = np.load(name, allow_pickle=True)
    
    # Transform into Dataframe for merge
    precomputed = pd.DataFrame(precomputed, columns=['index', 'label', 'mean_embed'])
    pos_ = pos.loc[precomputed.loc[precomputed['label']==1, 'index']]
    neg_ = neg.loc[precomputed.loc[precomputed['label']==-1, 'index']]
    tweets_ = pos_.append(neg_).reset_index()
    tweets_ = tweets_.merge(precomputed, how='inner', on=['index', 'label'], validate='1:1').set_index('index')
    print('Successfully loaded from file!')
    
except (FileNotFoundError) as e :
    print('Could not load word vectors from file...\nRecomputing word vectors...')
    # Create word vectors for the local dataset
    tweets_['mean_embed']= tweets_['tweet'].map(word_vector)

Trying to load word vectors from ../precomputed_data/word_vectors_100000_20.npy
Successfully loaded from file!
Wall time: 696 ms


In [50]:
tweets_

,tweet,label,mean_embed
index,,,
1020761,<user> kus niggas tryna find new ways to get f...,1,"[0.006111815659464072, 0.7197322757532972, -0...."
690958,joanna symons suggests how to find accommodati...,1,"[-0.017124226010274895, 0.4027337962952229, 0...."
656365,<user> what time ?,1,"[0.06875733450303735, 0.5183489266332838, 0.04..."
384680,<user> it was the right tweet son,1,"[0.09417493947047391, 0.5709764570942454, 0.04..."
393805,i wanna go . rt <user> finna go go buffalo wil...,1,"[0.10825893930221646, 0.5741484203775674, 0.20..."
...,...,...,...
210752,"novica blown wine glasses , ' crimson serpenti...",-1,"[0.06386397143353187, 0.5318056923844712, -0.0..."
97104,walthers ho scale hs #902074 ttwx all purpose ...,-1,"[0.09309077043951619, 0.5157496136248376, 0.12..."
975497,<--- ain't had sex in a long time . i'm gonna ...,-1,"[0.09434767776236758, 0.6438721760825019, 0.05..."


In [51]:
# Save word vectors file to gain time
word_vectors = tweets_[['label', 'mean_embed']].reset_index()
name = word_vectors_file_location+'word_vectors_'+str(local_t_size)+'_'+str(embeddings_dim_info)

np.save(name, word_vectors)

### Train-Test Split for our local dataset
We divide our local training set into a 75% training set and a 25% local testing set.

In [49]:
from sklearn.model_selection import train_test_split

train_, test_ = train_test_split(tweets_, test_size=0.25)
print(f"Local training set size : {train_.shape}.")
print(f"Local testing set size : {test_.shape}.")

Local training set size : (375000, 3).
Local testing set size : (125000, 3).


In [50]:
# Create features and label datasets
xtrain_, ytrain_ = train_.mean_embed.copy().tolist(), train_.label.copy().to_list()
xtest_, ytest_ = test_.mean_embed.copy().tolist(), test_.label.copy().tolist()

xtrain_[:1]

[[0.11209293083648302,
  0.8379536320000046,
  0.06435265794088334,
  0.28261244119135703,
  -0.4116261995677197,
  -0.0037634530456671076,
  -0.06068811384656975,
  -0.4951261722005948,
  -0.1770969131120541,
  -0.13442795039758862,
  -0.30454069901345904,
  0.2862791843832266,
  -0.2972696497099928,
  -0.010056398270032085,
  -0.05267474284327053,
  -0.16724661835906388,
  -0.031884223678409644,
  -0.11216669698455584,
  -0.4264506111642759,
  0.1936082316094817]]

### Pre-Processing
Here we compute our pre-processing on features.

In [51]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

def preprocess(X) :
    x=X.copy()
    
    # Standardize data
    standardizer=StandardScaler().fit_transform(x)
    
    # TODO Polynomial features and interactions
    
    # other data preprocessing
    
    return x

In [52]:
# Pre-process training set

xtrain_ = preprocess(xtrain_)
xtest_ = preprocess(xtest_)

### Estimating efficiency of model
Here we define metrics for model classification efficiency.
* Accuracy
* Precision
* Recall
* F1-Score

In [53]:
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

### Models Training
* Linear Regression
* Logistic Regression
* SVM
* Neural Networks

In [54]:
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

classifiers = dict()

In [55]:
%%time 
# Linear Regression
name = 'Linear Model'

linear_classifier = LinearRegression().fit(xtrain_, ytrain_)
score = linear_classifier.score(xtest_, ytest_)

classifiers[name] = (linear_classifier, score)

print(f"R2 score for {name} is {score}.")

R2 score for Linear Model is 0.013335540112904831.
Wall time: 1.6 s


In [56]:
%%time 
#### BASELINE : Logistic Regression

# Logistic Regression
name = 'Logistic Regression'

logistic_classifier = LogisticRegression().fit(xtrain_, ytrain_)
score = logistic_classifier.score(xtest_, ytest_)

classifiers[name] = (logistic_classifier, score)

print(f"R2 score for {name} is {score}.")

# Logistic Regression using Crossvalidation
name = 'Logistic Regression using cross-validation'

logisticCV_classifier = LogisticRegressionCV().fit(xtrain_, ytrain_)
score = logisticCV_classifier.score(xtest_, ytest_)

classifiers[name] = (logisticCV_classifier, score)

print(f"R2 score for Logistic Regression model using cross-validation is {score}.")

R2 score for Logistic Regression is 0.54616.
R2 score for Logistic Regression model using cross-validation is 0.546104.
Wall time: 17.1 s


In [57]:
# Compute predictions
predtest_ = logistic_classifier.predict(xtest_)

report = classification_report(ytest_, predtest_, labels=[1,-1], target_names=['pos', 'neg'], digits=5)
print(report)

              precision    recall  f1-score   support

         pos    0.53708   0.64278   0.58520     62256
         neg    0.55955   0.45029   0.49901     62744

    accuracy                        0.54616    125000
   macro avg    0.54832   0.54654   0.54210    125000
weighted avg    0.54836   0.54616   0.54194    125000



In [58]:
# Compute predictions
predtest_ = logisticCV_classifier.predict(xtest_)

report = classification_report(ytest_, predtest_, labels=[1,-1], target_names=['pos', 'neg'], digits=5)
print(report)

              precision    recall  f1-score   support

         pos    0.53703   0.64281   0.58518     62256
         neg    0.55950   0.45015   0.49890     62744

    accuracy                        0.54610    125000
   macro avg    0.54826   0.54648   0.54204    125000
weighted avg    0.54831   0.54610   0.54187    125000



In [59]:
%%time 
# Support Vector Machines
name = 'SVM classifier'

SVM_classifier = LinearSVC().fit(xtrain_, ytrain_)
score = SVM_classifier.score(xtest_, ytest_)

classifiers[name] = (SVM_classifier, score)

print(f"R2 score for {name} model is {score}.")

R2 score for SVM classifier model is 0.54628.
Wall time: 1min 6s


In [60]:
%%time
# Neural Network
name = 'Neural Network'

nn_classifier = MLPClassifier().fit(xtrain_,ytrain_)
score = nn_classifier.score(xtest_,ytest_)

classifiers[name] = (nn_classifier, score)

print(f"R2 score for {name} classifier is {score}.")

R2 score for Neural Network classifier is 0.582856.
Wall time: 3min 42s


In [61]:
# Compute predictions
predtest_ = nn_classifier.predict(xtest_)

report = classification_report(ytest_, predtest_, labels=[1,-1], target_names=['pos', 'neg'], digits=5)
print(report)

              precision    recall  f1-score   support

         pos    0.56868   0.67255   0.61627     62256
         neg    0.60318   0.49386   0.54307     62744

    accuracy                        0.58286    125000
   macro avg    0.58593   0.58320   0.57967    125000
weighted avg    0.58599   0.58286   0.57953    125000



---
## Testing
This section is dedicated to using the previous classifiers to predict the labels of the provided testing set.

In [62]:
# To format the testing data
def extract_tweet(tweet):
    return tweet.split(",", 1)[1]

In [63]:
## Loading data

# Load the testing data
test = pd.read_fwf(data_path+ 'test_data.txt', sep="\n", header=None)
test = test.rename(columns={0:'tweet', 1:'na1', 2:'na2'})

# Reformating it for submission
test.index = test.index+1 # Format asked by AI Crowd
test = test['tweet'].map(extract_tweet).to_frame()

test

,tweet
1,sea doo pro sea scooter ( sports with the port...
2,<user> shucks well i work all week so now i ca...
3,i cant stay away from bug thats my baby
4,<user> no ma'am ! ! ! lol im perfectly fine an...
5,"whenever i fall asleep watching the tv , i alw..."
...,...
9996,had a nice time w / my friend lastnite
9997,<user> no it's not ! please stop !
9998,not without my daughter ( dvd two-time oscar (...
9999,<user> have fun in class sweetcheeks


In [64]:
## Preparing data

# Create word vectors for tweets
test['mean_embed'] = test['tweet'].map(word_vector)

# Preprocess test data
xtest = preprocess(test.mean_embed.copy().tolist())

xtest[:1]

[[0.06600514685663708,
  0.582784556899372,
  -0.04151115815776958,
  0.13666445264367844,
  -0.1977931317665288,
  -0.016802999768011986,
  -0.038712791935644156,
  -0.2545744725436583,
  0.029000791532073186,
  0.06613123578801591,
  -0.14071631816239816,
  0.147312012718148,
  -0.2390666239629726,
  0.08578977417972455,
  -0.0034186956612259254,
  -0.052978385981814986,
  0.16010788866987907,
  -0.07031762344866747,
  -0.2815485402300905,
  0.16404086032326892]]

In [65]:
# Recalling classifiers 
# stored in format : 'classifier name'=(classifier, R2 score) 

print(f"Models computed so far are the following.\n ")
print(f"{'Classifier':50s} | {'R2 Score':20s}")
print(f"-----------------------------------------------------------------")
for k,v in classifiers.items() :
    print(f"{k:50s} | {v[1]:10.10f}")
print('\n')

Models computed so far are the following.
 
Classifier                                         | R2 Score            
-----------------------------------------------------------------
Linear Model                                       | 0.0133355401
Logistic Regression                                | 0.5461600000
Logistic Regression using cross-validation         | 0.5461040000
SVM classifier                                     | 0.5462800000
Neural Network                                     | 0.5828560000




In [66]:
# Making predictions
model = classifiers['Neural Network'][0]

predictions = model.predict(xtest)

In [67]:
# Creating submission file
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w', newline='') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})
            
create_csv_submission(test.index, predictions, '../submission/submission.csv')